In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import glob
import os

In [ ]:
STREET = "convention"

directory = f"data/{STREET}"
path_pattern = os.path.join(directory, "*.csv")
files = glob.glob(path_pattern)

dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append(df)

old_df = pd.concat(dfs, ignore_index=True)
print("Total number of rows: ", len(old_df))
print("Columns: ", list(df.columns))

In [ ]:
recent_df = pd.read_csv(f"data/comptages-routiers-permanents-{STREET}-filtered.csv", sep=";")
print("Total number of rows in recent data: ", len(recent_df))
print("Columns: ", list(recent_df.columns))

In [ ]:
# Create column mapping new name, recent name and old name
mapping = {
    "time": ["Date et heure de comptage", "t_1h"],
    "id_arc": ["Identifiant arc", "iu_ac"],
    "street": ["libelle", "Libelle"],
    "id_upstream_node": ["Identifiant noeud amont", "iu_nd_amont"],
    "upstream_node": ["Libelle noeud amon", "libelle_nd_amont"],
    "id_downstream_node": ["Identifiant noeud aval", "iu_nd_aval"],
    "downstream_node": ["Libelle noeud aval", "libelle_nd_aval"],
    "throughput": ["Débit horaire", "q"],
    "occupancy": ["Taux d'occupation", "k"],
}

# Alternatively, more clearly:
rename_old_df = {v[1]: k for k, v in mapping.items() if v[1] in old_df.columns}
rename_recent_df = {v[0]: k for k, v in mapping.items() if v[0] in recent_df.columns}

print(rename_old_df)
print(rename_recent_df)

# Rename the columns
old_df_renamed = old_df.rename(columns=rename_old_df)
recent_df_renamed = recent_df.rename(columns=rename_recent_df)

# Chzck if the columns are the same
print("Columns in old data: ", list(old_df_renamed.columns))
print("Columns in recent data: ", list(recent_df_renamed.columns))

In [ ]:
new_df = pd.concat([old_df_renamed, recent_df_renamed], ignore_index=True)

# Drop all columns that are not in the mapping
new_df = new_df[[k for k in mapping.keys() if k in new_df.columns]]

print("Total number of rows in new data: ", len(new_df))
print("Columns: ", list(new_df.columns))

In [ ]:
new_df.head()

In [ ]:
new_df.dtypes

In [ ]:
# Remove +00:00 in the time column of the new df
new_df["time"] = new_df["time"].str.replace("+00:00", "")

# Transform the time column to datetime
new_df["time"] = pd.to_datetime(new_df["time"])

# Sort the data by time
new_df = new_df.sort_values(by="time", ascending=True, ignore_index=True)

In [ ]:
new_df.head()

In [ ]:
new_df.info()

In [ ]:
no_dup = new_df.drop_duplicates()

print("Number of duplicates: ", len(new_df) - len(no_dup))

In [ ]:
no_dup.to_csv(f"data/{STREET}_merged.csv", index=False)